In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [3]:
unfilteredTrainData = getDataframe(2018)
unfilteredTrainData = addTokenizedColumnofTitle(unfilteredTrainData)

In [4]:
unfilteredTestData = getDataframe(2019)
unfilteredTestData = addTokenizedColumnofTitle(unfilteredTestData)
unfilteredTestData = unfilteredTestData.reset_index()

In [5]:
trainData = unfilteredTrainData.copy()
testData = unfilteredTestData.copy()

trainWords = getWordFrequencyDataframe(trainData,AllClasses)
priorProbabilities = getPriorProbabilities(trainData) 

trainWords = getWordFrequencyDataframe(trainData,AllClasses)
trainWords = obtainDataframeWithClassProbabilities(trainWords, AllClasses, delta, appendClassPrefix)
writeModel(trainWords,'model-2018.txt',AllClasses,appendClassPrefix)
trainWords = renameModelRows(trainWords, AllClasses, appendClassPrefix)
model = [trainWords, priorProbabilities]

testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'baseline-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])

In [6]:
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
print(df_confusion)

0.9913274787741865
predicted  ask_hn  show_hn   story
Post Type                         
ask_hn       5274        0     180
poll            1        0       5
show_hn        42     4457     404
story         308      250  126294


In [7]:
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)

predicted  ask_hn  show_hn   story  poll  precision    recall  F1_score
Post Type                                                              
ask_hn       5274        0     180     0   0.937600  0.966997  0.952071
poll            1        0       5     0   0.000000  0.000000  0.000000
show_hn        42     4457     404     0   0.946888  0.909035  0.927575
story         308      250  126294     0   0.995358  0.995601  0.995480


/anaconda3/envs/rz/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/anaconda3/envs/rz/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
